In [1]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns
gc.enable()

In [2]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00471/Data_for_UCI_named.csv

--2021-10-04 17:03:40--  https://archive.ics.uci.edu/ml/machine-learning-databases/00471/Data_for_UCI_named.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2417871 (2.3M) [application/x-httpd-php]
Saving to: ‘Data_for_UCI_named.csv.1’

Data_for_UCI_named. 100%[===================>]   2.31M  --.-KB/s    in 0.08s   

2021-10-04 17:03:40 (30.0 MB/s) - ‘Data_for_UCI_named.csv.1’ saved [2417871/2417871]



Split the data into an 80-20 train-test split with a random state of “1”. Use the standard scaler to transform the train set (x_train, y_train) and the test set (x_test). Use scikit learn to train a random forest and extra trees classifier. And use xgboost and lightgbm to train an extreme boosting model and a light gradient boosting model. Use random_state = 1 for training all models and evaluate on the test set.

Also, to improve the Extra Trees Classifier, you will use the following parameters (number of estimators, minimum number of samples, minimum number of samples for leaf node and the number of features to consider when looking for the best split) for the hyperparameter grid needed to run a Randomized Cross Validation Search (RandomizedSearchCV).

In [3]:
df = pd.read_csv('Data_for_UCI_named.csv')
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [4]:
# df['stabf'].replace('unstable', 0, inplace=True)
# df['stabf'].replace('stable', 1, inplace=True)
y = df['stabf'].copy()
df.drop(['stabf', 'stab'], axis=1, inplace=True)

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=1)

In [6]:
scaler = StandardScaler()

X_train = pd.DataFrame(data=scaler.fit_transform(X_train),
                       columns=X_train.columns)
X_test = pd.DataFrame(data=scaler.transform(X_test),
                      columns=X_test.columns)

In [7]:
from xgboost import XGBClassifier as xgbc
import lightgbm
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

In [8]:
n_estimators = [50, 100, 300, 500, 1000]
min_samples_split = [2, 3, 5, 7, 9]
min_samples_leaf = [1, 2, 4, 6, 8]
max_features = ['auto', 'sqrt', 'log2', None]
hyperparameter_grid = {'n_estimators': n_estimators, 
                       'min_samples_leaf': min_samples_leaf,
                       'min_samples_split': min_samples_split,
                       'max_features': max_features}

In [9]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics

### Question 14

What is the accuracy on the test set using the random forest classifier? In 4 decimal places.

In [10]:
rf_clf = RandomForestClassifier(random_state=1)
rf_clf.fit(X_train, y_train)
preds = rf_clf.predict(X_test)
print(round(metrics.accuracy_score(preds, y_test), 4))

0.929


### Question 15

What is the accuracy on the test set using the xgboost classifier? In 4 decimal places.

In [11]:
xgboost_clf = xgbc(random_state=1)
xgboost_clf.fit(X_train, y_train)

preds = xgboost_clf.predict(X_test)
round(metrics.accuracy_score(preds, y_test), 4)

0.9195

### Question 16

What is the accuracy on the test set using the LGBM classifier? In 4 decimal places.

In [12]:
lgb_clf = lightgbm.LGBMClassifier(random_state=1)
lgb_clf.fit(X_train, y_train)
preds = lgb_clf.predict(X_test)
round(metrics.accuracy_score(preds, y_test), 4)

0.9375

In [13]:
ext_clf = ExtraTreesClassifier(random_state=1)
ext_clf.fit(X_train, y_train)
preds = ext_clf.predict(X_test)
metrics.accuracy_score(preds, y_test)

0.928

Question 17

To improve the Extra Trees Classifier, you will use the following parameters (number of estimators, minimum number of samples, minimum number of samples for leaf node and the number of features to consider when looking for the best split) for the hyperparameter grid needed to run a Randomized Cross Validation Search (RandomizedSearchCV).

n_estimators = [50, 100, 300, 500, 1000]

min_samples_split = [2, 3, 5, 7, 9]

min_samples_leaf = [1, 2, 4, 6, 8]

max_features = ['auto', 'sqrt', 'log2', None]

hyperparameter_grid = {
  'n_estimators': n_estimators,
                   'min_samples_leaf': min_samples_leaf,
                   'min_samples_split': min_samples_split,
                   'max_features': max_features}

Using the ExtraTreesClassifier as your estimator with cv=5, n_iter=10, scoring = 'accuracy', n_jobs = -1, verbose = 1 and random_state = 1. What are the best hyperparameters from the randomized search CV?

In [14]:
tree = ExtraTreesClassifier(random_state=1)

cv = RandomizedSearchCV(tree,
                        hyperparameter_grid, scoring='accuracy',
                                n_iter=10,
                                cv=5,
                                n_jobs=-1,
                                verbose=1)

In [15]:
cv.fit(X_train, y_train)
preds = cv.predict(X_test)
metrics.accuracy_score(preds, y_test)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.7min finished


0.927

In [16]:
cv.best_params_

{'max_features': None,
 'min_samples_leaf': 8,
 'min_samples_split': 2,
 'n_estimators': 500}

### Question 18

Train a new ExtraTreesClassifier Model with the new Hyperparameters from the RandomizedSearchCV (with random_state = 1). Is the accuracy of the new optimal model higher or lower than the initial ExtraTreesClassifier model with no hyperparameter tuning?

In [20]:
new_ext_clf = cv.best_estimator_
new_ext_clf.fit(X_train, y_train)
preds = new_ext_clf.predict(X_test)
round(metrics.accuracy_score(preds, y_test), 4)

0.927

### Question 20

Find the feature importance using the optimal ExtraTreesClassifier model. Which features are the most and least important respectively?

In [19]:
import matplotlib.pyplot as plt

fi = pd.DataFrame(columns=['variable', 'value'])
fi['variable'] = X_train.columns
fi['value'] = new_ext_clf.feature_importances_
fi.sort_values(by='value', inplace=True)
fi

,variable,value
4,p1,0.003689
7,p4,0.004990
6,p3,0.005384
5,p2,0.005400
8,g1,0.102260
9,g2,0.106740
11,g4,0.110267
10,g3,0.112810
2,tau3,0.134675
3,tau4,0.135892
